In [1]:
# Tutorial for the scraping:
# https://www.youtube.com/watch?v=8dTpNajxaH0

import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
import datetime

# Define the start and end dates
start_date = datetime.datetime(2024, 11, 1)
end_date = datetime.datetime(2024, 11, 3)

# Create a list to store the URLs
urls = []

# Loop through the date range
current_date = start_date
while current_date <= end_date:
    # Format the date as DD.MM.YYYY
    formatted_date = current_date.strftime('%d.%m.%Y')
    # Create the URL
    url = f'https://www.eso.bg/imb/?imbalance-prices&date={formatted_date}'
    # Append the URL to the list
    urls.append(url)
    # Increment the current date by one day
    current_date += datetime.timedelta(days=1)



In [3]:
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

# table with prices from ESO
tablePrices = soup.find_all('table')[0]
listTitles = tablePrices.find_all('th')
listTitles

#get titles of the dataframe from ESO
df_Prices = pd.DataFrame(columns= [titles.text.strip() for titles in listTitles])

for url in urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    # table with prices from ESO
    tablePrices = soup.find_all('table')[0]
    listTitles = tablePrices.find_all('th')
    listTitles

    # find tr tag in table where prices are from ESO
    columnData  = tablePrices.find_all('tr')

    for row in columnData[1:]:
        row_data = row.find_all('td')
        individualRowData = [data.text.strip() for data in row_data]
        
        lenght = len(df_Prices)
        df_Prices.loc[lenght] = individualRowData


    # Strip to get the desired format "DD.MM.YYYY HH:MM"
    df_Prices['Сетълмент период'] = df_Prices['Сетълмент период'].str.split('-').str[0]
    df_Prices['Сетълмент период'] = df_Prices['Сетълмент период'].apply(lambda x: f"{x[:16]}")